In [1]:
import pandas as pd
from unidecode import unidecode

In [2]:
wiki = pd.read_csv("../datasets/wiki_movie_plots_deduped.csv")
wiki

,Release Year,Title,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
0,1901,Kansas Saloon Smashers,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Kansas_Saloon_Sm...,"A bartender is working at a saloon, serving dr..."
1,1901,Love by the Light of the Moon,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/Love_by_the_Ligh...,"The moon, painted with a smiling face hangs ov..."
2,1901,The Martyred Presidents,American,Unknown,NaN,unknown,https://en.wikipedia.org/wiki/The_Martyred_Pre...,"The film, just over a minute long, is composed..."
3,1901,"Terrible Teddy, the Grizzly King",American,Unknown,NaN,unknown,"https://en.wikipedia.org/wiki/Terrible_Teddy,_...",Lasting just 61 seconds and consisting of two ...
4,1902,Jack and the Beanstalk,American,"George S. Fleming, Edwin S. Porter",NaN,unknown,https://en.wikipedia.org/wiki/Jack_and_the_Bea...,The earliest known adaptation of the classic f...
...,...,...,...,...,...,...,...,...
34881,2014,The Water Diviner,Turkish,Director: Russell Crowe,Director: Russell Crowe\r\nCast: Russell Crowe...,unknown,https://en.wikipedia.org/wiki/The_Water_Diviner,"The film begins in 1919, just after World War ..."
34882,2017,Çalgı Çengi İkimiz,Turkish,Selçuk Aydemir,"Ahmet Kural, Murat Cemcir",comedy,https://en.wikipedia.org/wiki/%C3%87alg%C4%B1_...,"Two musicians, Salih and Gürkan, described the..."
34883,2017,Olanlar Oldu,Turkish,Hakan Algül,"Ata Demirer, Tuvana Türkay, Ülkü Duru",comedy,https://en.wikipedia.org/wiki/Olanlar_Oldu,"Zafer, a sailor living with his mother Döndü i..."
34884,2017,Non-Transferable,Turkish,Brendan Bradley,"YouTubers Shanna Malcolm, Shira Lazar, Sara Fl...",romantic comedy,https://en.wikipedia.org/wiki/Non-Transferable...,The film centres around a young woman named Am...


In [3]:
train = pd.read_csv("../datasets/gpt_data/train_plot_awards.csv", sep=',')
train

,_c0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,plot,awards
0,4,tt0010600,The Doll,Die Puppe,1919,\N,66,1898.0,True,The Doll is a 1919 silent film directed by E...,0
1,7,tt0011841,Way Down East,Way Down East,1920,\N,145,5376.0,True,The Way Down East is a 1920 silent film dire...,1
2,9,tt0012494,Déstiny,Der müde Tod,1921,\N,97,5842.0,True,Destiny is a 1921 German silent fantasy film...,0
3,25,tt0015163,The Navigator,The Navigator,1924,\N,59,9652.0,True,The Navigator is a 1924 silent comedy film s...,0
4,38,tt0016220,The Phantom of the Opera,The Phantom of the Opera,1925,\N,93,17887.0,True,The Phantom of the Opera is a 1925 silent ho...,0
...,...,...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,Trauma Center,NaN,2019,\N,87,12951.0,False,The Trauma Center is a 2019 drama film direc...,0
7955,9981,tt9741310,Slaxx,Slaxx,2020,\N,77,2464.0,False,Slaxx is a 2020 horror-comedy about a posses...,0
7956,9982,tt9742392,Kindred,Kindred,2020,\N,101,1719.0,False,The Kindred is a 2020 science fiction horror...,0
7957,9996,tt9850386,The Bee Gees: How Can You Mend a Broken Heart,NaN,2020,\N,111,4144.0,True,The Bee Gees: How Can You Mend a Broken Hear...,0


In [4]:
train["startYear"].replace(r"\N", None, inplace=True)
train["startYear"].fillna(train['endYear'], inplace=True)
train["startYear"] = train["startYear"].astype(int)

In [5]:
train["primaryTitle"] = train["primaryTitle"].str.lower()
train['primaryTitle'] = train['primaryTitle'].apply(unidecode)

In [3]:
wiki["Title"] = wiki["Title"].str.lower()
wiki.rename({"Title": "primaryTitle"}, axis=1, inplace=True)
wiki.rename({"Release Year": "startYear"}, axis=1, inplace=True)
wiki['primaryTitle'] = wiki['primaryTitle'].apply(unidecode)

In [4]:
# remove rows where the 'name' column is a duplicate
wiki = wiki.drop_duplicates(subset=['primaryTitle'], keep='last')

In [5]:
# find the duplicate rows based on all columns
wiki[wiki["primaryTitle"] == "interstellar"]

,startYear,primaryTitle,Origin/Ethnicity,Director,Cast,Genre,Wiki Page,Plot
21498,2014,interstellar,British,Director: Christopher Nolan,Director: Christopher Nolan\r\nCast: Matthew M...,unknown,https://en.wikipedia.org/wiki/Interstellar_(film),"In the mid-21st century, crop blights and dust..."


In [9]:
pd.merge(train, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

,_c0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,plot,awards,Genre
0,4,tt0010600,the doll,Die Puppe,1919,\N,66,1898.0,True,The Doll is a 1919 silent film directed by E...,0,NaN
1,7,tt0011841,way down east,Way Down East,1920,\N,145,5376.0,True,The Way Down East is a 1920 silent film dire...,1,NaN
2,9,tt0012494,destiny,Der müde Tod,1921,\N,97,5842.0,True,Destiny is a 1921 German silent fantasy film...,0,NaN
3,25,tt0015163,the navigator,The Navigator,1924,\N,59,9652.0,True,The Navigator is a 1924 silent comedy film s...,0,comedy
4,38,tt0016220,the phantom of the opera,The Phantom of the Opera,1925,\N,93,17887.0,True,The Phantom of the Opera is a 1925 silent ho...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,trauma center,NaN,2019,\N,87,12951.0,False,The Trauma Center is a 2019 drama film direc...,0,NaN
7955,9981,tt9741310,slaxx,Slaxx,2020,\N,77,2464.0,False,Slaxx is a 2020 horror-comedy about a posses...,0,NaN
7956,9982,tt9742392,kindred,Kindred,2020,\N,101,1719.0,False,The Kindred is a 2020 science fiction horror...,0,NaN
7957,9996,tt9850386,the bee gees: how can you mend a broken heart,NaN,2020,\N,111,4144.0,True,The Bee Gees: How Can You Mend a Broken Hear...,0,NaN


In [10]:
final_train = pd.merge(train, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")
final_train

,_c0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,label,plot,awards,Genre
0,4,tt0010600,the doll,Die Puppe,1919,\N,66,1898.0,True,The Doll is a 1919 silent film directed by E...,0,NaN
1,7,tt0011841,way down east,Way Down East,1920,\N,145,5376.0,True,The Way Down East is a 1920 silent film dire...,1,NaN
2,9,tt0012494,destiny,Der müde Tod,1921,\N,97,5842.0,True,Destiny is a 1921 German silent fantasy film...,0,NaN
3,25,tt0015163,the navigator,The Navigator,1924,\N,59,9652.0,True,The Navigator is a 1924 silent comedy film s...,0,comedy
4,38,tt0016220,the phantom of the opera,The Phantom of the Opera,1925,\N,93,17887.0,True,The Phantom of the Opera is a 1925 silent ho...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
7954,9966,tt9625664,trauma center,NaN,2019,\N,87,12951.0,False,The Trauma Center is a 2019 drama film direc...,0,NaN
7955,9981,tt9741310,slaxx,Slaxx,2020,\N,77,2464.0,False,Slaxx is a 2020 horror-comedy about a posses...,0,NaN
7956,9982,tt9742392,kindred,Kindred,2020,\N,101,1719.0,False,The Kindred is a 2020 science fiction horror...,0,NaN
7957,9996,tt9850386,the bee gees: how can you mend a broken heart,NaN,2020,\N,111,4144.0,True,The Bee Gees: How Can You Mend a Broken Hear...,0,NaN


In [11]:
final_train.to_csv("../datasets/gpt_data/train_plots_awards_genre_no_duplicates.csv", index=False)

### Validation

In [6]:
validation = pd.read_csv("../datasets/gpt_data/validation_plots_awards.csv", sep=',')
validation

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards
0,0,tt0003740,Cabiria,NaN,1914,\N,148,3452.0,Cabiria is a 1914 Italian silent film direct...,1
1,1,tt0008663,A Man There Was,Terje Vigen,1917,\N,65,1882.0,A Man There Was is a 1917 silent film direct...,0
2,3,tt0010307,J'accuse!,NaN,1919,\N,166,1692.0,The film J'accuse! (1919) is a French silent...,0
3,18,tt0014429,Safety Last!,Safety Last!,1923,\N,74,19898.0,Safety Last! is a 1923 silent comedy film st...,0
4,27,tt0015175,Die Nibelungen: Siegfried,NaN,1924,\N,143,5676.0,The epic saga of Die Nibelungen: Siegfried f...,0
...,...,...,...,...,...,...,...,...,...,...
950,9974,tt9686154,You Will Die at 20,NaN,2019,\N,103,2106.0,You Will Die at 20 is a 2019 Sudanese drama ...,0
951,9976,tt9690328,Pápér Spidérs,Paper Spiders,2020,\N,109,NaN,The Pápér Spidérs is a 2020 drama film direc...,0
952,9980,tt9735790,Me You Madness,Me You Madness,2021,\N,98,1056.0,Me You Madness is a 2021 romantic comedy abo...,0
953,9984,tt9769668,Tughlaq Durbar,Tughlaq Durbar,2021,\N,145,1430.0,The Tughlaq Durbar is a 2021 Indian Tamil-la...,0


In [7]:
validation["startYear"].replace(r"\N", None, inplace=True)
validation["startYear"].fillna(validation['endYear'], inplace=True)
validation["startYear"] = validation["startYear"].astype(int)

In [8]:
validation["primaryTitle"] = validation["primaryTitle"].str.lower()
validation['primaryTitle'] = validation['primaryTitle'].apply(unidecode)

In [9]:
pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,0,tt0003740,cabiria,NaN,1914,\N,148,3452.0,Cabiria is a 1914 Italian silent film direct...,1,NaN
1,1,tt0008663,a man there was,Terje Vigen,1917,\N,65,1882.0,A Man There Was is a 1917 silent film direct...,0,NaN
2,3,tt0010307,j'accuse!,NaN,1919,\N,166,1692.0,The film J'accuse! (1919) is a French silent...,0,NaN
3,18,tt0014429,safety last!,Safety Last!,1923,\N,74,19898.0,Safety Last! is a 1923 silent comedy film st...,0,comedy
4,27,tt0015175,die nibelungen: siegfried,NaN,1924,\N,143,5676.0,The epic saga of Die Nibelungen: Siegfried f...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
950,9974,tt9686154,you will die at 20,NaN,2019,\N,103,2106.0,You Will Die at 20 is a 2019 Sudanese drama ...,0,NaN
951,9976,tt9690328,paper spiders,Paper Spiders,2020,\N,109,NaN,The Pápér Spidérs is a 2020 drama film direc...,0,NaN
952,9980,tt9735790,me you madness,Me You Madness,2021,\N,98,1056.0,Me You Madness is a 2021 romantic comedy abo...,0,NaN
953,9984,tt9769668,tughlaq durbar,Tughlaq Durbar,2021,\N,145,1430.0,The Tughlaq Durbar is a 2021 Indian Tamil-la...,0,NaN


In [10]:
validation_final = pd.merge(validation, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")
validation_final

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,0,tt0003740,cabiria,NaN,1914,\N,148,3452.0,Cabiria is a 1914 Italian silent film direct...,1,NaN
1,1,tt0008663,a man there was,Terje Vigen,1917,\N,65,1882.0,A Man There Was is a 1917 silent film direct...,0,NaN
2,3,tt0010307,j'accuse!,NaN,1919,\N,166,1692.0,The film J'accuse! (1919) is a French silent...,0,NaN
3,18,tt0014429,safety last!,Safety Last!,1923,\N,74,19898.0,Safety Last! is a 1923 silent comedy film st...,0,comedy
4,27,tt0015175,die nibelungen: siegfried,NaN,1924,\N,143,5676.0,The epic saga of Die Nibelungen: Siegfried f...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
950,9974,tt9686154,you will die at 20,NaN,2019,\N,103,2106.0,You Will Die at 20 is a 2019 Sudanese drama ...,0,NaN
951,9976,tt9690328,paper spiders,Paper Spiders,2020,\N,109,NaN,The Pápér Spidérs is a 2020 drama film direc...,0,NaN
952,9980,tt9735790,me you madness,Me You Madness,2021,\N,98,1056.0,Me You Madness is a 2021 romantic comedy abo...,0,NaN
953,9984,tt9769668,tughlaq durbar,Tughlaq Durbar,2021,\N,145,1430.0,The Tughlaq Durbar is a 2021 Indian Tamil-la...,0,NaN


In [11]:
validation_final.to_csv("../datasets/gpt_data/validation_plots_awards_genre_no_duplicates.csv", index=False)

### Test data

In [6]:
test = pd.read_csv("../datasets/gpt_data/test_plots_awards.csv")

In [7]:
test["startYear"].replace(r"\N", None, inplace=True)
test["startYear"].fillna(test['endYear'], inplace=True)
test["startYear"] = test["startYear"].astype(int)
test["primaryTitle"] = test["primaryTitle"].str.lower()
test['primaryTitle'] = test['primaryTitle'].apply(unidecode)

In [8]:
test

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0
...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0


In [9]:
pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0,drama
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0,western
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0,NaN
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0,NaN
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0,NaN
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0,NaN
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0,NaN
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0,NaN


In [10]:
test_final = pd.merge(test, wiki[["primaryTitle", "Genre", "startYear"]],on=["primaryTitle", "startYear"], how="left")
test_final

,Unnamed: 0,tconst,primaryTitle,originalTitle,startYear,endYear,runtimeMinutes,numVotes,plot,awards,Genre
0,22,tt0014972,he who gets slapped,He Who Gets Slapped,1924,\N,95,3654.0,He Who Gets Slapped is a 1924 silent film di...,0,drama
1,23,tt0015016,the iron horse,NaN,1924,\N,150,2136.0,The Iron Horse is a 1924 American silent wes...,0,western
2,26,tt0015174,die nibelungen: kriemhild's revenge,NaN,1924,\N,129,4341.0,The Die Nibelungen: Kriemhild's Revenge is a...,0,NaN
3,28,tt0015214,at 3:25,NaN,1925,1925,59,1724.0,At 3:25 is a drama film about a young woman ...,0,NaN
4,34,tt0015863,go west,NaN,1925,\N,69,4188.0,Go West is a 1925 silent comedy film starrin...,0,NaN
...,...,...,...,...,...,...,...,...,...,...,...
1081,9942,tt9430698,one piece: stampede,NaN,2019,\N,101,5109.0,The Straw Hat Pirates are invited to the Pir...,0,NaN
1082,9943,tt9441638,the big ugly,NaN,2020,\N,106,5780.0,The Big Ugly is a 2020 crime drama set in th...,0,NaN
1083,9948,tt9495690,pagalpanti,Pagalpanti,2019,\N,149,2331.0,The Págálpánti is a 2019 Indian drama film d...,0,NaN
1084,9950,tt9519642,the wedding unplanner,NaN,2020,\N,110,NaN,The Wedding Unplanner is a 2020 romantic com...,0,NaN


In [11]:
test_final.to_csv("../datasets/gpt_data/test_plots_awards_genre_no_duplicates.csv", index=False)